**The goal is to predict the number of confirmed cases and fatalities for different regions.** See the “Data” tab from the competition website for more details. The scores are calculated based on the root mean squared logarithmic error (log error metrics penalize % differences rather than absolute differences, so a predicted 101 instead of 100 is penalized equally to 1,010,000 instead of 1 million - even though one error is 10,000 times larger in absolute terms) 

In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
import time
import datetime
from datetime import datetime
from datascience import *

import plotly.express as px
import plotly.graph_objects as go

import warnings
warnings.filterwarnings('ignore')

In [3]:
test = pd.read_csv("test.csv")
test

,ForecastId,Province_State,Country_Region,Date
0,1,NaN,Afghanistan,2020-04-02
1,2,NaN,Afghanistan,2020-04-03
2,3,NaN,Afghanistan,2020-04-04
3,4,NaN,Afghanistan,2020-04-05
4,5,NaN,Afghanistan,2020-04-06
...,...,...,...,...
13454,13455,NaN,Zimbabwe,2020-05-10
13455,13456,NaN,Zimbabwe,2020-05-11
13456,13457,NaN,Zimbabwe,2020-05-12
13457,13458,NaN,Zimbabwe,2020-05-13


In [4]:
train = pd.read_csv("train.csv")
train

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities
0,1,NaN,Afghanistan,2020-01-22,0.0,0.0
1,2,NaN,Afghanistan,2020-01-23,0.0,0.0
2,3,NaN,Afghanistan,2020-01-24,0.0,0.0
3,4,NaN,Afghanistan,2020-01-25,0.0,0.0
4,5,NaN,Afghanistan,2020-01-26,0.0,0.0
...,...,...,...,...,...,...
25974,35647,NaN,Zimbabwe,2020-04-09,11.0,3.0
25975,35648,NaN,Zimbabwe,2020-04-10,13.0,3.0
25976,35649,NaN,Zimbabwe,2020-04-11,14.0,3.0
25977,35650,NaN,Zimbabwe,2020-04-12,14.0,3.0


In [5]:
sum_df = pd.pivot_table(train, values=['ConfirmedCases','Fatalities'], index=['Date'],aggfunc=np.sum)
display(sum_df.max())
sum_df

ConfirmedCases    2019143.0
Fatalities         119481.0
dtype: float64

,ConfirmedCases,Fatalities
Date,,
2020-01-22,554.0,17.0
2020-01-23,653.0,18.0
2020-01-24,939.0,26.0
2020-01-25,1432.0,42.0
2020-01-26,2113.0,56.0
...,...,...
2020-04-09,1595174.0,95449.0
2020-04-10,1691542.0,102519.0
2020-04-11,1771337.0,108497.0


In [7]:
train_data_by_country = train.groupby(['Date','Country_Region'],as_index=False).agg({'ConfirmedCases': 'sum', 'Fatalities': 'sum'})#, 'GrowthRate':'last' })
max_train = train['Date'].max()
train_data_by_country_confirm = train_data_by_country.query('(Date == @max_train) & (ConfirmedCases > 100)').sort_values('ConfirmedCases', ascending=False)
train_data_by_country_confirm.set_index('Country_Region', inplace=True)
train_data_by_country_confirm.style.background_gradient(cmap='Oranges').format({'ConfirmedCases': "{:.0f}"})#, 'GrowthRate': "{:.2f}"})

,Date,ConfirmedCases,Fatalities
Country_Region,,,
US,2020-04-13,682456,23527
Spain,2020-04-13,170099,17756
Italy,2020-04-13,159516,20465
France,2020-04-13,137875,14986
Germany,2020-04-13,130072,3194
United Kingdom,2020-04-13,89570,11347
China,2020-04-13,83213,3345
Iran,2020-04-13,73303,4585
Turkey,2020-04-13,61049,1296
